<a href="https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Visual Notebook to Using BERT for the First TIme.ipynb



<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" />

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the text. The text is a list of sentences from film reviews. And we will classify each sentence as either speaking "positively" about its subject or "negatively".

## Models: Sentence Sentiment Classification
Our goal is to create a model that takes a sentence (just like the ones in our dataset) and produces either 1 (indicating the sentence carries a positive sentiment) or a 0 (indicating the sentence carries a negative sentiment). We can think of it as looking like this:

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Dataset
The dataset we will use in this example is [SST2](https://nlp.stanford.edu/sentiment/index.html), which contains sentences from movie reviews, each labeled as either positive (has the value 1) or negative (has the value 0):


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>

## Installing the transformers library
Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [4]:
df = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small.jsonl", lines = True)

In [5]:
df.tail()

,id,fandoms,pair
52596,3926a1bd-6d33-5513-a694-12d7590443d4,"[True Blood, Austin & Ally]",[Enjoy! And time stands still beneath the air ...
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,"[True Blood, Kuroko no Basuke/黒子のバスケ]","[""I forgive you,"" she blurted out looking stra..."
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,"[Austin & Ally, Five Nights at Freddy´s]","[""What? Why? What""s happening? Where are we go..."
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,"[Five Nights at Freddy´s, Austin & Ally]","[""Gah!"" I growled. ""I need a password to acces..."
52600,900c4a19-e22b-59fd-9cc7-8490919b1696,"[Five Nights at Freddy´s, Austin & Ally]",[Chica heard a noise below her and saw Golden ...


In [ ]:
df['pair'][0][0]

For performance reasons, we'll only use 2,000 sentences from the dataset

In [90]:
batch_1 = pd.concat([df[:2000], df[-2000:]])
batch_1.tail()

,id,fandoms,pair
52596,3926a1bd-6d33-5513-a694-12d7590443d4,"[True Blood, Austin & Ally]",[Enjoy! And time stands still beneath the air ...
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,"[True Blood, Kuroko no Basuke/黒子のバスケ]","[""I forgive you,"" she blurted out looking stra..."
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,"[Austin & Ally, Five Nights at Freddy´s]","[""What? Why? What""s happening? Where are we go..."
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,"[Five Nights at Freddy´s, Austin & Ally]","[""Gah!"" I growled. ""I need a password to acces..."
52600,900c4a19-e22b-59fd-9cc7-8490919b1696,"[Five Nights at Freddy´s, Austin & Ally]",[Chica heard a noise below her and saw Golden ...


In [93]:
for p, pair in enumerate(batch_1['pair']):
    for i, text in enumerate(pair):
        pair[i] = text[0:250]
    batch_1['pair'][p] = ' [SEP] '.join(pair)


In [94]:
batch_1['pair'][0]

'I shift a bit, warily letting my eyes dart from one owl to the other -- but my eyes are trained on the Barn Owl the most. Like Hoole...so like Hoole... He turns a bit, and our eyes meet directly. I can"t describe it...in this next moment, I don"t loo [SEP] "All will become one with Russia," he said, almost simply, his cheer eerie. Fists were already clenched; now they groped about, for a pan, a rifle, a sword-there was nothing. In some way, this brought her but a sigh of relief-Gilbert and Roderich, sh'

In [8]:
df_labels = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small-truth.jsonl", lines = True)

In [22]:
batch_1_labels = pd.concat([df_labels[:2000], df_labels[-2000:]])
batch_1_labels['same'] = batch_1_labels['same'].astype(int) 
batch_1_labels.head()

,id,same,authors
0,6cced668-6e51-5212-873c-717f2bc91ce6,1,"[1446633, 1446633]"
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,1,"[1446633, 1446633]"
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,1,"[1446633, 1446633]"
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,1,"[1446633, 1446633]"
4,4fe541af-912e-5a86-81a5-94c6d3891509,1,"[1446633, 1446633]"


We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [23]:
batch_1_labels['same'].value_counts()

1    2000
0    2000
Name: same, dtype: int64

In [24]:
len(batch_1) + len(batch_1_labels)

8000

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [13]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 28.0kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 317kB/s]  
Downloading: 100%|██████████| 455k/455k [00:01<00:00, 263kB/s] 
Downloading: 100%|██████████| 570/570 [00:00<00:00, 645kB/s]
Downloading: 100%|██████████| 420M/420M [00:50<00:00, 8.71MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from

In [113]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [102]:
tokenized = batch_1['pair'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [95]:
batch_1['pair'][0]

'I shift a bit, warily letting my eyes dart from one owl to the other -- but my eyes are trained on the Barn Owl the most. Like Hoole...so like Hoole... He turns a bit, and our eyes meet directly. I can"t describe it...in this next moment, I don"t loo [SEP] "All will become one with Russia," he said, almost simply, his cheer eerie. Fists were already clenched; now they groped about, for a pan, a rifle, a sword-there was nothing. In some way, this brought her but a sigh of relief-Gilbert and Roderich, sh'

In [100]:
enc_seq = tokenizer.encode(batch_1['pair'][0])


In [79]:
enc_seq_list = enc_seq['input_ids'][0] + enc_seq['input_ids'][1]

In [107]:
tokenizer.decode(tokenized[0])

'[CLS] i shift a bit, warily letting my eyes dart from one owl to the other - - but my eyes are trained on the barn owl the most. like hoole... so like hoole... he turns a bit, and our eyes meet directly. i can " t describe it... in this next moment, i don " t loo [SEP] " all will become one with russia, " he said, almost simply, his cheer eerie. fists were already clenched ; now they groped about, for a pan, a rifle, a sword - there was nothing. in some way, this brought her but a sigh of relief - gilbert and roderich, sh [SEP]'

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [108]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

print(max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

305


Our dataset is now in the `padded` variable, we can view its dimensions below:

In [109]:
np.array(padded).shape

(4000, 305)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [110]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(4000, 305)

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [123]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

input_ids = input_ids.to('cuda:0')
attention_mask = attention_mask.to('cuda:0')
model = model.to('cuda:0')

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

RuntimeError: CUDA out of memory. Tried to allocate 3.49 GiB (GPU 0; 16.00 GiB total capacity; 10.92 GiB already allocated; 3.43 GiB free; 10.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [33]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [34]:
labels = batch_1[1]

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [35]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-train-test-split-sentence-embedding.png" />

### [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [38]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scores:  0.8160000000000001


We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [41]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

<img src="https://jalammar.github.io/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [42]:
lr_clf.score(test_features, test_labels)

0.84

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [43]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.521 (+/- 0.00)


So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.